In [42]:
import torch
import json
from tqdm import tqdm

from classic_link_pred import read_lp_data

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ENTITY_FEATURES_FILE = '../data/wikidata5m_inductive/entity_description_first_sentence_embedding.pt'
ENTITY_2_ID_FILE = '../data/wikidata5m_inductive/uri_to_id.json'
RELATION_2_ID_FILE = '../data/wikidata5m_inductive/relation_uri_to_id.json'

In [12]:
with open(ENTITY_2_ID_FILE) as uri_to_id_in:
    uri_to_id = json.load(uri_to_id_in)

with open(RELATION_2_ID_FILE) as uri_to_id_in:
    relation_uri_to_id = json.load(uri_to_id_in)

In [11]:
entity_features = torch.load(ENTITY_FEATURES_FILE)

In [13]:
data_train = read_lp_data(f'../data/wikidata5m_inductive/wikidata5m_inductive_train.txt', uri_to_id=uri_to_id,
                                  relation_uri_to_id=relation_uri_to_id, data_sample='train')

Preprocessing: train


In [31]:
def negative_sampling(edge_index, num_nodes, eta=50):
    # Sample edges by corrupting either the subject or the object of each edge.
    mask_1 = torch.rand(edge_index.size(0) * eta) < 0.5
    mask_2 = ~mask_1

    mask_1 = mask_1
    mask_2 = mask_2

    neg_edge_index = edge_index.clone().repeat(eta, 1)
    neg_edge_index[mask_1, 0] = torch.randint(num_nodes, (1, mask_1.sum()))
    neg_edge_index[mask_2, 1] = torch.randint(num_nodes, (1, mask_2.sum()))

    return neg_edge_index


eta = 1
edge_idxs_neg = negative_sampling(data_train.edge_index.T, len(uri_to_id.keys()), eta=eta)
edge_type_neg = data_train.edge_type.repeat(eta)

#out_pos = model.forward(edge_idxs[:, 0], relation_idx, edge_idxs[:, 1])
#out_neg = model.forward(edge_idxs_neg[:, 0], relation_idx.repeat(eta), edge_idxs_neg[:, 1])

In [43]:
edge_idxs_neg.T.size()

negative_input = []

for i in tqdm(range(edge_idxs_neg.size(0))):
    negative_input.append(torch.cat([entity_features[edge_idxs_neg[i][0]], entity_features[edge_idxs_neg[i][1]]]))


 15%|█▌        | 3101537/20496514 [02:03<11:31, 25166.13it/s] 

KeyboardInterrupt



In [ ]:
gt = torch.cat([torch.ones(len(relation_idx)), torch.zeros(len(relation_idx) * eta)], dim=0)


In [3]:
from sklearn.neural_network import MLPClassifier
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(500, 500, 2), random_state=1)

clf.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(500, 500, 2), random_state=1,
              solver='lbfgs')

In [4]:
clf.predict([[2., 2.], [-1., -2.]])

array([1, 1])